<a href="https://colab.research.google.com/github/nahumsa/VAE-RBM-Pretraining/blob/Entanglement/VAE%20Entanglement%20Witness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating entangled and separable states with quti

## Installing dependencies

In [0]:
!pip install qutip

## Creating the dataset

We want to take the partial trace of the density matrix and check if it $\rho > 0$. 

Here we are following the Perez-Horondecki criterion, if the partial transpose of the density matrix is negative, the state has to be entangled. 

Because it the state is separable, it will be positive definite by definition.

In [0]:
from itertools import product
import qutip as qutip

def measurement(density_matrix, base, name_base):
  """Measuring the quantum state on a given basis.
  """
  _measurements_names = []
  _measurements = []

  for (name_1, meas_1),(name_2,meas_2) in product(zip(name_basis, basis),zip(name_basis, basis)):
    measurement_op = qutip.tensor(meas_1,meas_2)
    _measurements.append(qutip.expect(measurement_op, density_matrix))
    _measurements_names.append(name_1 + name_2)

  return [i for i in zip(_measurements, _measurements_names)]

def create_density_matrix(base,name_base):
  """ Create a random density matrix."""

  for (name_1, basis_1),(name_2, basis_2) in product(zip(name_basis, basis),zip(name_basis, basis)):
    if name_1 == 'I' and name_2 == 'I':
      density_matrix = 0.25*qutip.tensor(qutip.identity(2), qutip.identity(2))
    else: 
      density_matrix += 0.25*np.random.rand()*qutip.tensor(basis_1,basis_2)
  
  return density_matrix

def create_dataset(n_samples):
  
  _states = []
  _labels = []
  _measurements = []

  #Basis Measured
  name_basis = ['I','X', 'Y', 'Z']
  basis = [qutip.identity(2), qutip.sigmax(),qutip.sigmay(),qutip.sigmaz()]



  for _ in range(n_samples):
    density = create_density_matrix(base=basis, name_base=name_basis)
  
    #Partial Trace
    part_density = density.ptrace(1)
  
    #Labels: 1 if entangled -1 if separable
    if (part_density.eigenenergies() < 0).any():
      _labels.append(1)
  
    else:
      _labels.append(-1)  

    _states.append(density)  
  
    val_measurements = measurement(density_matrix=density, 
                                   base=basis, 
                                   name_base=name_basis)
  
    _measurements.append(val_measurements)
    
  return _states, _measurements, _labels

In [0]:
samples = 100
states, measurements, labels = create_dataset(samples)

# Build a Variational Autoencoder